Es geht los mit ein paar Imports und dann mit der Funktion, die die Rufnummern aufbereitet. Dabei werden in diesem Beispiel alle Nummern, die nicht aus Deutschland sind und die nicht mit einer "0" bzw. der Länderkennung für Deutschland beginnen (+49, 0049) aussortiert.

Wenn dies nicht gewünscht ist, muss diese Funktion angepasst werden.

In [32]:
import datetime
import pandas as pd
import random

def formatMobilePhoneNumber(number : str) -> str :
    number = number.replace(' ', '')
    if len(number) == 0: return None

    if number.startswith('+') and not number.startswith('+49'):
        return None
    if number.startswith('00') and not number.startswith('0049'):
        return None
    # replace leading +49 or 0049 with 0
    if number.startswith('+49'):
        number = "0" + number[3:]
    if number.startswith('0049'):
        number = "0" + number[4:]
    # take only digits
    number = "".join([c for c in number if c.isdigit()])
    return number
    

def samplePrint (number):
    print (number + " --> " + str(formatMobilePhoneNumber(number)))

samplePrint("+49 123 456 789")
samplePrint("0049 123 456 789")
samplePrint("123 456 789")
samplePrint("123456789")
samplePrint("+33 123 456 789")
samplePrint("0033 123 456 789")

+49 123 456 789 --> 0123456789
0049 123 456 789 --> 0123456789
123 456 789 --> 123456789
123456789 --> 123456789
+33 123 456 789 --> None
0033 123 456 789 --> None


Hier erzeugen wir einen Beispieldatensatz, im echten Leben würde man
die oberste Zeile nutzen (auskommentiert) und die Daten aus einer Exceldatei laden.

In [33]:

# df = pd.read_excel('sample.xlsx')

# create sample data with colums Vorname, Nachname, Telefon1, Telefon2
df = pd.DataFrame({
    'Vorname': ['Max', 'Moritz', 'Peter'], 
    'Nachname': ['Mustermann', 'Muster', 'Meier'], 
    'Telefon1': ['+49 3456789', '0987654321', ''], 
    'Telefon2': ['+33 123456789', '3987654321', '2123123123']})


#df = pd.read_excel('sample.xlsx')
df.head()


,Vorname,Nachname,Telefon1,Telefon2
0,Max,Mustermann,+49 3456789,+33 123456789
1,Moritz,Muster,0987654321,3987654321
2,Peter,Meier,,2123123123


Dann erzeugen wir die Nachrichten für jeden Empfänger, in dem Beispiel werden die Nachrichten
mittels den Einträgen der übrigen Spalten personalisiert.

Außerdem erzeugen wir eine Spalte mit einem Anzeigenamen (display), so dass wir später die
Nachrichten leichter zuordnen können. (Der Automateflow zeigt den Displaynamen während des
Versendens an.)

Die Versandreihenfolge der Nachrichten wird aus der Sortierung nach dem Anzeigenamen abgeleitet.

In [34]:
# create message for each row
df['message1'] = df.apply(lambda x: f"Hallo {x['Vorname']} {x['Nachname']}, dies ist eine erste Message.", axis=1)
df['message2'] = df.apply(lambda x: f"Hallo {x['Vorname']} {x['Nachname']}, dies ist eine zweite Message.", axis=1)
df['message3'] = df.apply(lambda x: f"Hallo {x['Vorname']} {x['Nachname']}, dies ist eine dritte Message.", axis=1)
df['display'] = df.apply(lambda x: f"{x['Nachname']}, {x['Vorname']}", axis=1)

df.head()

,Vorname,Nachname,Telefon1,Telefon2,message1,message2,message3,display
0,Max,Mustermann,+49 3456789,+33 123456789,"Hallo Max Mustermann, dies ist eine Message.",Max Mustermann,Mustermann,"Mustermann, Max"
1,Moritz,Muster,0987654321,3987654321,"Hallo Moritz Muster, dies ist eine Message.",Moritz Muster,Muster,"Muster, Moritz"
2,Peter,Meier,,2123123123,"Hallo Peter Meier, dies ist eine Message.",Peter Meier,Meier,"Meier, Peter"


Hier wird für jede Telefonnummer zu jeder Message ein Versandeintrag erstellt.
Die Spaltennamen müssen auf die Namen der Spaltennamen im importierten Datensatz 
umgestellt werden.

In [35]:
# Anpassung der Spaltennamen
numberColumns = ['Telefon1', 'Telefon2']
messageColumns = ['message1', 'message2', 'message3']
displayColumn = 'display'
# Erzeugung der Ziel-Dataframes
target_frames = []
counter = 0
for numberColumn in numberColumns:
    for messageColumn in messageColumns:
        counter += 1
        target_df = df[[numberColumn, messageColumn, displayColumn]] \
            .rename(columns={
                numberColumn: 'number', 
                messageColumn: 'message', 
                displayColumn: 'display'
                })
        target_df['order'] = counter            
        target_frames.append(target_df)
target_frame = pd.concat(target_frames, ignore_index=True)
target_frame['number'] = target_frame['number'].apply(formatMobilePhoneNumber)
target_frame.drop_duplicates(inplace=True)
target_frame.dropna(subset=['number', 'message'], inplace=True)
# sort by number and order
target_frame.sort_values(by=['display', 'order'], inplace=True)
# add a column with a random number from 5 to 12
target_frame['delay'] = target_frame['display'].apply(lambda x: random.randint(5, 12))
# create sortable datestamp
stamp = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
filename = stamp + 'sms'+stamp+'.json'
target_frame.to_json(filename, orient='records', indent=4)
print ("JSON abgespeichert in " + filename)
# display the new dataframe
target_frame.head()

JSON abgespeichert in 2023-11-16-09-40-43sms2023-11-16-09-40-43.json


,number,message,display,order,delay
11,2123123123,"Hallo Peter Meier, dies ist eine Message.","Meier, Peter",4,9
14,2123123123,Peter Meier,"Meier, Peter",5,11
17,2123123123,Meier,"Meier, Peter",6,12
1,0987654321,"Hallo Moritz Muster, dies ist eine Message.","Muster, Moritz",1,5
4,0987654321,Moritz Muster,"Muster, Moritz",2,8
